In [1]:
import random
import os

import numpy as np
import pandas as pd

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import missingno

# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)

seed = 42 # seed 값 설정
random.seed(seed) # 파이썬 난수 생성기
os.environ['PYTHONHASHSEED'] = str(seed) # 해시 시크릿값 고정
np.random.seed(seed) # 넘파이 난수 생성기

torch.manual_seed(seed) # 파이토치 CPU 난수 생성기
torch.backends.cudnn.deterministic = True # 확정적 연산 사용 설정
torch.backends.cudnn.benchmark = False   # 벤치마크 기능 사용 해제
torch.backends.cudnn.enabled = False        # cudnn 기능 사용 해제

if device == 'cuda':
    torch.cuda.manual_seed(seed) # 파이토치 GPU 난수 생성기
    torch.cuda.manual_seed_all(seed) # 파이토치 멀티 GPU 난수 생성기

cpu


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/data/test.csv')

In [4]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43
...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,64.74
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,77.37
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,70.14
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,91.51


In [5]:
!pip install rdkit-pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 23.8 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors

def calculate_metabolic_stability_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    logP = Descriptors.MolLogP(mol)
    # 화합물의 친유성을 측정한 것으로 지질 또는 비극성 환경에서의 용해도를 나타냅니다. 생물학적 막을 통과하는 화합물의 능력을 반영합니다.
    apka = Descriptors.MolWt(mol)
    # 화합물의 산 해리 상수의 추정치로 다양한 pH 조건에서 이온화 거동에 대한 정보를 제공합니다.
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    # 화합물에서 회전 가능한 결합의 수입니다. 이것은 화합물의 유연성과 효소 또는 다른 분자와의 잠재적인 상호 작용에 대한 통찰력을 제공할 수 있습니다.
    num_heteroatoms = Descriptors.NumHeteroatoms(mol)
    # 분자 내 헤테로원자(탄소 및 수소 이외의 원자) 수. 이는 화합물의 반응성과 대사 안정성에 영향을 줄 수 있습니다.
    num_hydrogen_acceptors = Descriptors.NumHAcceptors(mol)
    # 분자 내 수소 결합 수용체의 수. 이들은 결합 및 반응성에 영향을 미치는 다른 분자의 수소 결합 기증자와 상호 작용할 수 있는 사이트입니다.
    num_hydrogen_donors = Descriptors.NumHDonors(mol)
    # 분자 내 수소 결합 기증자의 수입니다. 이들은 수소 결합 상호작용에서 수소 원자를 제공할 수 있는 사이트입니다.
    # morgan_fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    morgan_fingerprint = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    # 분자 하위 구조의 이진 벡터 표현입니다. 이 열에는 화합물과 효소의 상호 작용 및 대사 안정성에 영향을 줄 수 있는 구조적 특징을 포착하는 이진 지문이 포함되어 있습니다.
    morgan_array = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(morgan_fingerprint, morgan_array)

    return logP, apka, num_rotatable_bonds, num_heteroatoms, num_hydrogen_acceptors, num_hydrogen_donors, morgan_array

train[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_heteroatoms',
    'num_hydrogen_acceptors', 'num_hydrogen_donors', 'morgan_fingerprint'
]] = train['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

test[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_heteroatoms',
    'num_hydrogen_acceptors', 'num_hydrogen_donors', 'morgan_fingerprint'
]] = test['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

train


,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,logP,apka,num_rotatable_bonds,num_heteroatoms,num_hydrogen_acceptors,num_hydrogen_donors,morgan_fingerprint
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37,3.87744,400.504,8,8,6,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47,3.35474,301.415,2,5,4,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45,1.20450,297.366,3,7,7,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60,3.89356,494.665,5,9,7,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43,2.81772,268.316,1,4,3,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,64.74,2.74730,396.200,4,11,5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,77.37,2.27630,359.389,3,7,5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,70.14,2.04130,261.325,5,5,5,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,91.51,1.42720,284.699,4,7,6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [7]:
train['AlogP'].fillna(train['AlogP'].median(), inplace=True)
test['AlogP'].fillna(test['AlogP'].median(), inplace=True)

In [8]:
class CustomDataset(Dataset):
    def __init__(self, data, target_col=None, transform=None, is_test=False):
        self.is_test = is_test
        self.transform = transform
        self.is_test = is_test

        if not self.is_test:
            self.data = self.transform.fit_transform(np.stack(data['morgan_fingerprint']))
        else: # test
            self.data = self.transform.transform(np.stack(data['morgan_fingerprint']))

        if target_col is not None and not self.is_test:
            self.target = data[target_col]

    def __getitem__(self, index):
        features = self.data[index]

        if hasattr(self, 'target'):
            target = self.target[index]
            return torch.tensor(features).to(device).float(), torch.tensor(target).to(device).float().unsqueeze(dim=-1)
        else:
            return torch.tensor(features).to(device).float()

    def __len__(self):
        return len(self.data)


In [13]:
transform = VarianceThreshold(threshold=0.05)

train_MLM = CustomDataset(train, target_col='MLM', transform=transform, is_test=False)
train_HLM = CustomDataset(train, target_col='HLM', transform=transform, is_test=False)

input_size = train_MLM.data.shape[1]
input_size

251

In [22]:
train_HLM.data

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int8)

In [16]:
# train,valid split
train_MLM_dataset, valid_MLM_dataset = train_test_split(train_MLM, test_size=0.2, random_state=42)
train_HLM_dataset, valid_HLM_dataset = train_test_split(train_HLM, test_size=0.2, random_state=42)

In [18]:
torch.tensor(train_MLM.data[1]).shape, torch.tensor(train['MLM'][1]).float().unsqueeze(dim=-1).shape

(torch.Size([251]), torch.Size([1]))

In [64]:
# Hyperparameter
CFG = {'BATCH_SIZE': 256,
       'EPOCHS': 1000,
       'HIDDEN_SIZE': 251,
       'OUTPUT_SIZE': 1,
       'DROPOUT_RATE': 0.7,
       'FEATURE_SIZE': 251,
       'NUM_LAYERS' : 4,
       'LEARNING_RATE': 0.001
      }

In [65]:
train_MLM_loader = DataLoader(dataset=train_MLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=True)

valid_MLM_loader = DataLoader(dataset=valid_MLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=False)


train_HLM_loader = DataLoader(dataset=train_HLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=True)

valid_HLM_loader = DataLoader(dataset=valid_HLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=False)

In [66]:
X_train, y_train = next(iter(train_MLM_loader))
print (X_train.shape, y_train.shape)

torch.Size([256, 251]) torch.Size([256, 1])


In [67]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, feature_size, hidden_size, num_layers, dropout_p, output_size):
        super(LSTMModel, self).__init__()

        self.sequenceclassifier = nn.LSTM(
                input_size=feature_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True,
                bidirectional=True,
                dropout=dropout_p
        )
        # (256, 1, 251) -> (256, 1, hidden_size * 2)

        self.fc = nn.Sequential(
            nn.ReLU(),  # LeakyReLU 대신 ReLU 사용
            nn.Linear(hidden_size * 2, output_size)
        )
        # (256, hidden_size * 2) -> (256, output_size)

    def forward(self, x):
        # |x| = (256, 1, 251)
        output, _ = self.sequenceclassifier(x)  # |output| = (256, 1, hidden_size * 2)
        output = output[:, -1, :]  # |output| = (256, hidden_size * 2)
        y = self.fc(output)  # (256, output_size)
        return y


In [68]:
model_MLM = LSTMModel(CFG['FEATURE_SIZE'],CFG['HIDDEN_SIZE'],CFG['NUM_LAYERS'],CFG['DROPOUT_RATE'],CFG['OUTPUT_SIZE']).to(device)
model_HLM = LSTMModel(CFG['FEATURE_SIZE'],CFG['HIDDEN_SIZE'],CFG['NUM_LAYERS'],CFG['DROPOUT_RATE'],CFG['OUTPUT_SIZE']).to(device)

In [69]:
import torch.nn as nn

class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()  # 기존의 MSELoss 함수 사용

    def forward(self, output, target):
        mse_loss = self.mse(output, target)  # 기존의 MSELoss를 계산
        rmse_loss = torch.sqrt(mse_loss)  # MSE에 제곱근 씌워 RMSE 계산
        return rmse_loss

criterion = RMSELoss()
optimizer_MLM = torch.optim.Adam(model_MLM.parameters(), lr=CFG['LEARNING_RATE'])
optimizer_HLM = torch.optim.Adam(model_HLM.parameters(), lr=CFG['LEARNING_RATE'])


In [70]:
def train(train_loader, valid_loader, model, criterion, optimizer, epochs, patience=100):
    best_valid_loss = float('inf')
    no_improvement_count = 0

    for epoch in range(epochs):
        model.train()  # 모델을 훈련 모드로 설정
        running_loss = 0
        for inputs, targets in train_loader:

            inputs = inputs.unsqueeze(1)  # (256, 251) -> (256, 1, 251)

            optimizer.zero_grad()

            output = model(inputs)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()  # 모델을 검증 모드로 설정
        valid_loss = 0
        with torch.no_grad():
          for inputs, targets in valid_loader:

            inputs = inputs.unsqueeze(1)  # (256, 251) -> (256, 1, 251)

            output = model(inputs)
            loss = criterion(output, targets)
            valid_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        avg_valid_loss = valid_loss / len(valid_loader)
        print(f'Epoch: {epoch}/{epochs}, Train Loss: {avg_train_loss}, Valid Loss: {avg_valid_loss}')

        if avg_valid_loss < best_valid_loss:
          best_valid_loss = avg_valid_loss
          no_improvement_count = 0
          best_model_state = model.state_dict()
        else:
          no_improvement_count += 1
          if no_improvement_count >= patience:
            print(f'얼리 스토핑: {patience} 에포크 동안 검증 손실이 향상되지 않음. 에포크 {epoch}에서 훈련 중단.')
            break

    # 최적의 모델 상태 불러오기
    model.load_state_dict(best_model_state)

    return model

In [71]:
print("Training Start: MLM")
model_MLM = train(train_MLM_loader, valid_MLM_loader, model_MLM, criterion, optimizer_MLM, epochs=CFG['EPOCHS'])

print("Training Start: HLM")
model_HLM = train(train_HLM_loader, valid_HLM_loader, model_HLM, criterion, optimizer_HLM, epochs=CFG['EPOCHS'])

Training Start: MLM
Epoch: 0/1000, Train Loss: 50.903778769753195, Valid Loss: 50.56126149495443
Epoch: 1/1000, Train Loss: 47.585411765358664, Valid Loss: 46.77966054280599
Epoch: 2/1000, Train Loss: 44.65989615700462, Valid Loss: 44.80829747517904
Epoch: 3/1000, Train Loss: 42.87137707796964, Valid Loss: 43.24353154500326
Epoch: 4/1000, Train Loss: 41.364351445978336, Valid Loss: 41.89552307128906
Epoch: 5/1000, Train Loss: 40.12920518354936, Valid Loss: 40.75764846801758
Epoch: 6/1000, Train Loss: 39.0369873046875, Valid Loss: 39.82798767089844
Epoch: 7/1000, Train Loss: 38.191538377241656, Valid Loss: 39.05725351969401
Epoch: 8/1000, Train Loss: 37.472902818159625, Valid Loss: 38.43890889485677
Epoch: 9/1000, Train Loss: 36.88442542336204, Valid Loss: 37.96820068359375
Epoch: 10/1000, Train Loss: 36.478118896484375, Valid Loss: 37.60636647542318
Epoch: 11/1000, Train Loss: 36.16472660411488, Valid Loss: 37.33470662434896
Epoch: 12/1000, Train Loss: 35.926597595214844, Valid Loss: 3

In [72]:
torch.save(model_MLM.state_dict(), '/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/Many-to-one_LSTM_Model_MLM_1.pth')  # 모델 객체의 state_dict 저장
torch.save(model_HLM.state_dict(), '/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/Many-to-one_LSTM_Model_HLM_1.pth')

In [73]:
model_MLM.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/Many-to-one_LSTM_Model_MLM_1.pth'))
model_HLM.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/Many-to-one_LSTM_Model_HLM_1.pth'))

<All keys matched successfully>

In [78]:
test_MLM = CustomDataset(test, target_col=None, transform=transform, is_test=True)
test_HLM = CustomDataset(test, target_col=None, transform=transform, is_test=True)

test_MLM_loader = DataLoader(dataset=test_MLM,
                             batch_size=CFG['BATCH_SIZE'],
                             shuffle=False)

test_HLM_loader = DataLoader(dataset=test_HLM,
                             batch_size=CFG['BATCH_SIZE'],
                             shuffle=False)

In [81]:
def inference(test_loader, model):
    model.eval()
    preds = []

    with torch.no_grad():
        for inputs in test_loader:

            inputs = inputs.unsqueeze(1)  # (256, 251) -> (256, 1, 251)

            output = model(inputs)
            preds.extend(output.cpu().numpy().flatten().tolist())

    return preds

In [82]:
predictions_MLM = inference(test_MLM_loader, model_MLM)
predictions_HLM = inference(test_HLM_loader, model_HLM)

In [83]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/data/sample_submission.csv')

submission['MLM'] = predictions_MLM
submission['HLM'] = predictions_HLM
submission

,id,MLM,HLM
0,TEST_000,15.006695,61.335728
1,TEST_001,61.088417,70.075821
2,TEST_002,57.311104,87.030212
3,TEST_003,23.964069,72.702866
4,TEST_004,53.175064,80.534409
...,...,...,...
478,TEST_478,34.738350,64.757706
479,TEST_479,92.616318,90.107727
480,TEST_480,36.332047,79.415115
481,TEST_481,32.527252,96.071335


In [84]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/submissions/Many-to-one_LSTM_Model_1_submission.csv', index=False)